In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### 导入数据

In [2]:
data_train = pd.read_csv('E:\\tianchi\\titanic\\train.csv')
data_test = pd.read_csv('E:\\tianchi\\titanic\\test.csv')

### 初探数据 

In [3]:
data_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [5]:
data_train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


#### 1. 对于Age用均值填充  2.对于Cabin，由于丢失太多，故舍  3. 对于 PassengerId ,对目标无影响，故舍

In [28]:
#对train和test集都进行年龄处理
data_train['Age'] = data_train['Age'].fillna(data_train['Age'].median())
data_test['Age'] = data_test['Age'].fillna(data_test['Age'].median())

#对测试集的Fare进行填充
data_test['Fare'] = data_test['Fare'].fillna(data_test.groupby('Pclass')['Fare'].mean()[3])

### 线性回归模型 

In [30]:
# 线性回归
from sklearn.linear_model import LinearRegression

#交叉验证
from sklearn.model_selection import KFold

In [55]:
# 选取简单的特征【船舱等级，年龄，兄弟个数，父母小孩个数，船票价格】
predictors = ["Pclass","Age","SibSp","Parch","Fare"]

# 线性回归
lr = LinearRegression()

#样本平均分成3份，3折交叉验证
kf = KFold(n_splits=3,shuffle=False,random_state=1)

#设定一个空list,用于存放3次的预测值
predictions = []

for train,test in kf.split(data_train):
    #选取训练特征和label
    train_predictors = data_train[predictors].iloc[train,:]
    train_target = data_train['Survived'].iloc[train]
    
    #开始训练
    lr.fit(train_predictors,train_target)
    
    #预测
    test_predictions = lr.predict(data_train[predictors].iloc[test,:])
    
    #将预测结果装入predictions中
    predictions.append(test_predictions)

#### 上述的 kf.split()方法将训练集平均分成3份，其中两份作为训练集，一份作为测试集，其中，测试集的顺序是由前到后的，所以，接下来的三次预测结果合并的时候正好相当于对每个样本都预测了一遍，并且是按照顺序的。

In [52]:
#上一步的predictions中为三个数组，分别是3次交叉验证得到的预测结果，concatenate函数的作用是将之合并成为一个array
predictions = np.concatenate(predictions,axis=0)

In [53]:
#将概率改为0--1
predictions[predictions > 0.5] = 1
predictions[predictions <= 0.5] = 0

In [54]:
#计算准确率，即预测对的总个数除以样本数
accuary = sum(predictions[predictions == data_train['Survived']])/len(predictions)
print (accuary)

0.16610549943883277


### 逻辑回归

In [46]:
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression

In [57]:
#初始化逻辑回归算法
LogisReg = LogisticRegression(random_state = 1)

#训练
LogisReg.fit(data_train[predictors],data_train['Survived'])

#使用sklearn库里面的交叉验证函数获取预测准确率分数
scores = model_selection.cross_val_score(LogisReg,data_train[predictors],data_train["Survived"],cv=3)
 
#使用交叉验证分数的平均值作为最终的准确率
print("准确率为: ",scores.mean())

准确率为:  0.6992143658810326


### 增加特征Sex和Embarked来预测，看看是否有影响

In [69]:
# 性别没有缺失，对性别进行处理，meal用0，female用1
data_train.loc[data_train['Sex'] == 'male','Sex'] = 0
data_train.loc[data_train['Sex'] == 'female','Sex'] = 1

In [73]:
# 对于港口Embarked，先统计信息,发现S港口上船的人最多，就用S填充
data_train.groupby('Embarked')['Embarked'].count()

Embarked
C    168
Q     77
S    644
Name: Embarked, dtype: int64

In [74]:
#对于港口Embarked缺失，用S填充，并将港口替换为数字
data_train['Embarked'] = data_train['Embarked'].fillna('S')
data_train.loc[data_train['Embarked'] == 'C','Embarked'] = 0
data_train.loc[data_train['Embarked'] == 'Q','Embarked'] = 1
data_train.loc[data_train['Embarked'] == 'S','Embarked'] = 2

In [80]:
#增加两个特征，用逻辑回归算法进行预测
predictors = ["Pclass","Sex","Age","SibSp","Parch","Fare","Embarked"]

LogisticReg = LogisticRegression(random_state = 1)
LogisticReg.fit(data_train[predictors],data_train['Survived'])

scores = model_selection.cross_val_score(LogisticReg,data_train[predictors],data_train['Survived'],cv = 3)

print ("Accuracy:",scores.mean())

Accuracy: 0.7923681257014591
